In [12]:
from matplotlib import pyplot as plt
from rasterio.mask import mask
from rasterio.features import rasterize
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
import numpy as np
import mpld3
import os

In [4]:
%matplotlib inline
mpld3.enable_notebook()
plt.rcParams["figure.figsize"] = (9,9)

In [5]:
name = "Lacey"

# Navigate to the study site data directory
# This directory should contain NAIP, CHM, and boundary files
os.chdir(f"../data/{name}")
current_dir = os.getcwd()
print(f"Working directory: {current_dir}")

Working directory: c:\Users\jvonnonn\Gitlab_repos\meadows\data\Lacey


In [10]:
naip_path = f"naip_{name}.tif"
chm_path = f"chm_{name}.tif"
wm_path = f"wm_{name}.tif"
mc_path = f"main_channel_{name}.geojson"
meadow_extent_path = f"meadow_extent.geojson"

In [32]:
with rio.open(chm_path) as src:
    chm_data = src.read()
    chm_transform = src.transform
    chm_shape = (src.height, src.width)
    chm_profile = src.profile

chm_data = np.squeeze(chm_data)  # Remove single-dimensional entries
#chm_data = np.where(chm_data < 0, 0, chm_data)  # Set negative values to zero

with rio.open(wm_path) as src:
    water_mask = src.read()
    wm_profile = src.profile

water_mask = np.squeeze(water_mask)
# Read NAIP raster and meadow extent
with rio.open(naip_path) as naip_src:
    naip_crs = naip_src.crs

meadow_gdf = gpd.read_file(meadow_extent_path)

mc = gpd.read_file(mc_path)

# Reproject meadow extent to CHM CRS if needed
if mc.crs != chm_profile["crs"]:
    mc = mc.to_crs(chm_profile["crs"])

mc['geometry'] = mc.geometry.buffer(15)

shapes = ((geom, 1) for geom in mc.geometry)

main_channel = rasterize(
    shapes=shapes,
    out_shape=chm_shape,
    transform=chm_transform,
    fill=0,
    dtype='uint8'
)

# Reproject meadow extent to NAIP CRS if needed
if meadow_gdf.crs != naip_crs:
    meadow_gdf = meadow_gdf.to_crs(naip_crs)

meadow_geom = meadow_gdf.geometry.values

# Clip NAIP image to meadow extent
with rio.open(naip_path) as src:
    naip_data, naip_clipped_transform = mask(src, meadow_geom, crop=True, nodata=0)
    naip_data = np.transpose(naip_data, (1, 2, 0))  # (rows, cols, bands)

naip_data_float = naip_data.astype(np.float32)
naip_data_float[naip_data_float == 0] = np.nan  # Set zero values to NaN for calculations
naip_data = np.where(naip_data == 0, np.nan, naip_data)  # Set zero values to NaN

NAIP band order = R,G,B,NIR

In [15]:
R = naip_data[:,:,0]
G = naip_data[:,:,1]
B = naip_data[:,:,2]
NIR = naip_data[:,:,3]

In [16]:
RGB = np.dstack((R, G, B)).astype(np.uint8)

C:\Users\jvonnonn\AppData\Local\Temp\1\ipykernel_5220\2104806323.py:1: RuntimeWarning: invalid value encountered in cast
  RGB = np.dstack((R, G, B)).astype(np.uint8)


In [ ]:
# uint8 is the standard 8-bit per channel integer format (0-255) used by most display libraries.
# matplotlib.imshow treats:
#  - uint8 arrays as 0..255 color values per channel,
#  - float arrays as 0.0..1.0 color values per channel (values outside are clipped).
# If your image is stored as floats but not normalized to 0..1, casting to uint8 (or normalizing then casting)
# ensures colors render as intended and avoids silent clipping/contrast issues.
#
# Example: normalize the first three NAIP bands to 0..255 and display as uint8 RGB.

rgb_float = naip_data_float[..., :3]  # (rows, cols, 3) float image with NaNs
# Replace NaNs with 0 (black) for display; choose different fill if desired
rgb_float = np.where(np.isnan(rgb_float), 0.0, rgb_float)

# Heuristic: detect whether floats are already in 0..1 or in 0..255 range and convert accordingly
p99 = np.nanpercentile(rgb_float, 99)
if p99 <= 1.0:
    # floats in 0..1 -> scale to 0..255
    rgb_uint8 = (np.clip(rgb_float, 0.0, 1.0) * 255).astype(np.uint8)
else:
    # floats likely already in 0..255 or other DN range -> rescale by percentile stretch to preserve contrast
    p1 = np.nanpercentile(rgb_float, 1)
    p99 = max(p99, p1 + 1e-6)
    rgb_uint8 = (np.clip((rgb_float - p1) / (p99 - p1), 0.0, 1.0) * 255).astype(np.uint8)

plt.imshow(rgb_uint8)
plt.title("RGB rendered as uint8")
plt.axis("off")
plt.show()

In [18]:
# using NIR, R, and B bands
NDVI = (NIR - R) / (NIR + R)
NDWI = (G - NIR) / (G + NIR)
brightness = (R + G + B)
avg = (NIR + R + B) / 3
bri_avg = (R - avg) + (G - avg) + (B - avg) / avg
bri_avg2 = ((R - avg) + (G - avg) + (B - avg)) / avg

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(11, 11), sharex=True, sharey=True)
ax[0, 0].imshow(NDVI, cmap='RdYlGn')
ax[0, 0].set_title('NDVI')
ax[0, 1].imshow(NDWI, cmap='RdYlGn')
ax[0, 1].set_title('NDWI')
ax[0, 2].imshow(brightness, cmap='gray')
ax[0, 2].set_title('Brightness')
ax[1, 0].imshow(avg, cmap='gray')
ax[1, 0].set_title('Average Reflectance')
ax[1, 1].imshow(bri_avg, cmap='gray')
ax[1, 1].set_title('Brightness - Average / Average')
ax[1, 2].imshow(bri_avg2, cmap='gray')
ax[1, 2].set_title('Brightness - Average divided by Average')
plt.tight_layout()
plt.show();

In [ ]:
np.nanpercentile(NDWI, 100)

In [ ]:
np.nanpercentile(bri_avg2, 100)

In [ ]:
water =  (NDWI > 0.3) & (bri_avg2 > 0.5) #0.3 and 0.5 for Lacey
plt.imshow(water)

In [21]:
np.nanpercentile(brightness, 100)

np.float32(585.53064)

In [22]:
np.unique(wm_data)

array([0, 1], dtype=uint8)

In [23]:
np.unique(water)

array([False,  True])

In [ ]:
#bare_mask = (brightness > 320) &(chm_data < 0.5) #& (bri_avg > 29)
bare_mask = (brightness > 475) & (chm_data < 0.5) & ~water & (water_mask != 1) & (main_channel != 1) & (NDVI < 0.05)  # Adjusted condition to include NDVI threshold
bare_classified = np.zeros(chm_data.shape, dtype=np.uint8)
bare_classified[bare_mask] = 1  # Class 1 for bare
#bare_classified[~bare_mask] = 2  # Class 2 for non-bare
bare_classified = np.expand_dims(bare_classified, axis=0)  # Add channel dimension

In [ ]:
plt.imshow(bare_classified[0])

In [27]:
with rio.open("bare_classified.tif", "w", **chm_profile) as dst:
    dst.write(bare_classified)

# ID trees using tree height to crown diameter ratio--- for future work 

In [28]:

from scipy.ndimage import maximum_filter, label, find_objects
from skimage.feature import peak_local_max
from rasterio.plot import show


In [ ]:
#chm_data[chm_data < 4] = 0


# Detect local maxima (tree tops)
coordinates = peak_local_max(
    chm_data,
    min_distance=3,  # minimum distance between peaks (in pixels)
    threshold_abs=4,  # minimum height to be considered a tree
    #indices=True
)

# Plot CHM and detected tree tops
plt.figure(figsize=(10, 8))
#plt.imshow(chm_data, cmap='viridis')
plt.imshow(rgb_uint8)
plt.scatter(coordinates[:, 1], coordinates[:, 0], c='red', s=0.025, label='Tree Tops', alpha=0.8)
plt.title('Detected Tree Tops from CHM')
plt.show()



## Sumary -%cover:  
trees, shrubs, bare earth

In [ ]:
with rio.open('bare_classified_clipped.tif') as src:
    bare_classified_clipped = src.read()
    pixel = src.transform.a
    bare_earth_area = np.sum(bare_classified_clipped == 1) * pixel**2

In [ ]:
with rio.open("CHM_clipped.tif") as src:
    chm_data = src.read()
    pixel = src.transform.a
    trees = np.sum(chm_data > 5) * pixel**2
    shrubs = np.sum(((chm_data > 0.2) & (chm_data < 5))) * pixel**2
    total_area = np.sum(~np.isnan(chm_data)) * pixel**2

In [ ]:
bare_earth_area

In [ ]:
round(bare_earth_area / total_area * 100, 2)

In [ ]:
round(trees / total_area * 100, 2)

In [ ]:
round(shrubs / total_area * 100, 2)